# BQNT Carry Execution - Daily Signals

Main notebook for daily signal generation and portfolio construction.

**Run this notebook daily to generate trading signals.**

## 1. Setup

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# BQL
import bql
bq = bql.Service()

# Add src to path
import sys
sys.path.insert(0, '../src')

print(f"Daily Signals - {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# Import carry execution modules
from config.tickers import (
    TRADING_UNIVERSE, FX_SPOT_TICKERS, FX_FORWARD_TICKERS,
    METALS_TICKERS, VOLATILITY_TICKERS, get_bloomberg_ticker
)
from config.parameters import SIGNAL_WEIGHTS, REGIME_PARAMS
from data.bql_loader import BQLDataLoader
from data.cache import get_cache, reset_cache
from signals.carry import CarrySignalEngine
from signals.momentum import MomentumSignalEngine
from signals.regime import RegimeSignalEngine
from portfolio.construction import PortfolioConstructor
from reporting.daily import DailyReportGenerator

print("Modules loaded successfully!")

In [ ]:
# Initialize components
cache = get_cache()
loader = BQLDataLoader(cache=cache)
carry_engine = CarrySignalEngine()
momentum_engine = MomentumSignalEngine()
regime_engine = RegimeSignalEngine()
portfolio = PortfolioConstructor()
reporter = DailyReportGenerator()

print("Components initialized!")

## 2. Fetch Data

In [ ]:
# Fetch FX data (spot and forward)
print("Fetching FX data...")
fx_data = loader.fetch_fx_data(days=252)
spot_prices = fx_data['spot']
forward_prices = fx_data['forward']

print(f"  Spot prices: {spot_prices.shape}")
print(f"  Forward prices: {forward_prices.shape}")

In [ ]:
# Fetch metals data
print("Fetching metals data...")
metals_prices = loader.fetch_metals_data(days=252)
print(f"  Metals prices: {metals_prices.shape}")

In [ ]:
# Fetch volatility data
print("Fetching volatility data...")
vol_data = loader.fetch_volatility_data(days=252)
print(f"  Volatility data: {vol_data.shape}")

# Get current VIX
current_vix = vol_data['VIX'].iloc[-1] if 'VIX' in vol_data.columns else 20.0
print(f"\nCurrent VIX: {current_vix:.2f}")

In [ ]:
# Combine all prices for momentum calculation
all_prices = pd.concat([
    spot_prices,
    metals_prices[['XAUUSD', 'XAGUSD']] if 'XAUUSD' in metals_prices.columns else pd.DataFrame()
], axis=1)

print(f"\nCombined prices: {all_prices.shape}")
print(f"Assets: {list(all_prices.columns)}")

## 3. Calculate Signals

In [ ]:
# Calculate momentum signals
print("Calculating momentum signals...")
momentum_signals = momentum_engine.calculate_signals(all_prices)
latest_momentum = momentum_engine.get_latest_signals(all_prices)

print("\nLatest Momentum Signals:")
print(latest_momentum.round(3))

In [ ]:
# Calculate carry signals (FX only)
print("Calculating carry signals...")

# Calculate volatility for normalization
returns = all_prices.pct_change()
volatility = returns.rolling(21).std() * np.sqrt(252)

carry_signals = carry_engine.calculate_signals(
    spot_prices,
    forward_prices,
    volatility
)
latest_carry = carry_engine.get_latest_signals(spot_prices, forward_prices, volatility)

print("\nLatest Carry Signals:")
print(latest_carry.round(3))

In [ ]:
# Detect regime
print("Detecting regime...")
regime_info = regime_engine.calculate_combined_regime(current_vix)

print(f"\nRegime Analysis:")
print(f"  VIX Level: {regime_info['vix_level']:.2f}")
print(f"  VIX Regime: {regime_info['vix_regime'].upper()}")
print(f"  Position Multiplier: {regime_info['combined_multiplier']:.2f}x")

## 4. Construct Portfolio

In [ ]:
# Get regime multiplier series
if 'VIX' in vol_data.columns:
    _, regime_multiplier = regime_engine.calculate_regime_series(vol_data['VIX'])
else:
    regime_multiplier = pd.Series(1.0, index=all_prices.index)

# Extend carry signals to all assets (metals get 0 carry)
carry_extended = carry_signals.reindex(columns=all_prices.columns).fillna(0)

# Construct portfolio
print("Constructing portfolio...")
weights = portfolio.construct_portfolio(
    momentum_signals=momentum_signals,
    carry_signals=carry_extended,
    returns=returns,
    regime_multiplier=regime_multiplier,
)

latest_weights = weights.iloc[-1]

print("\nPortfolio Weights:")
print(latest_weights.round(3))

In [ ]:
# Portfolio summary
gross_exposure = latest_weights.abs().sum()
net_exposure = latest_weights.sum()
long_exposure = latest_weights[latest_weights > 0].sum()
short_exposure = latest_weights[latest_weights < 0].sum()

print("\nExposure Summary:")
print(f"  Gross: {gross_exposure:.1%}")
print(f"  Net: {net_exposure:+.1%}")
print(f"  Long: {long_exposure:.1%}")
print(f"  Short: {short_exposure:.1%}")

## 5. Generate Report

In [ ]:
# Display daily report
reporter.display_report(
    momentum_signals=latest_momentum,
    carry_signals=latest_carry.reindex(latest_momentum.index).fillna(0),
    weights=latest_weights,
    regime_info=regime_info,
)

## 6. Signal Visualization

In [ ]:
# Plot signals heatmap
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Momentum signals (last 20 days)
ax1 = axes[0]
momentum_recent = momentum_signals.tail(20)
im1 = ax1.imshow(momentum_recent.T, aspect='auto', cmap='RdYlGn', vmin=-1, vmax=1)
ax1.set_title('Momentum Signals (Last 20 Days)')
ax1.set_yticks(range(len(momentum_recent.columns)))
ax1.set_yticklabels(momentum_recent.columns)
plt.colorbar(im1, ax=ax1)

# Current weights
ax2 = axes[1]
colors = ['green' if w > 0 else 'red' for w in latest_weights]
ax2.barh(range(len(latest_weights)), latest_weights, color=colors)
ax2.set_yticks(range(len(latest_weights)))
ax2.set_yticklabels(latest_weights.index)
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Current Portfolio Weights')
ax2.set_xlabel('Weight')

# VIX and regime
ax3 = axes[2]
if 'VIX' in vol_data.columns:
    vol_data['VIX'].tail(60).plot(ax=ax3, color='purple')
    ax3.axhline(y=12, color='green', linestyle='--', alpha=0.5, label='Low')
    ax3.axhline(y=20, color='blue', linestyle='--', alpha=0.5, label='Normal')
    ax3.axhline(y=25, color='orange', linestyle='--', alpha=0.5, label='Elevated')
    ax3.axhline(y=30, color='red', linestyle='--', alpha=0.5, label='High')
    ax3.set_title('VIX (Last 60 Days)')
    ax3.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 7. Position Details

In [ ]:
# Detailed position table
position_df = pd.DataFrame({
    'Weight': latest_weights,
    'Momentum': latest_momentum.reindex(latest_weights.index).fillna(0),
    'Carry': latest_carry.reindex(latest_weights.index).fillna(0),
    'Direction': latest_weights.apply(lambda x: 'LONG' if x > 0.01 else ('SHORT' if x < -0.01 else 'FLAT')),
}).round(3)

# Sort by absolute weight
position_df = position_df.reindex(latest_weights.abs().sort_values(ascending=False).index)

print("\nPosition Details (sorted by weight):")
display(position_df)

In [ ]:
# Cache statistics
print(f"\nCache Statistics:")
print(f"  Size: {cache.size} entries")
print(f"  Hit Rate: {cache.stats['hit_rate']:.1%}")

---

**End of Daily Signal Report**

Run `02_backtest.ipynb` for historical performance analysis.